# <font color=purple>Práctica 2 de Análisis Numérico </font>

## <font color=purple> Interpolación y Reglas de Cuadratura</font>

### Nombre de los integrantes

## Ejercicio 1  $\quad$   <font color = blue>*HistoSpline*</font>

Considera la siguiente tabla con las frecuencias absolutas
de una variable aleatoria

$$\begin{array}{lr} 
 \text{intervalo} & \text{frecuencia} \\
 & \text{absoluta} \\
 [-3.5,-3] & 3 \\
 [-3,-2.5] & 6 \\
 [-2.5,-2] & 31 \\
 [-2,-1.5] & 80 \\
 [-1.5,-1] & 192 \\
 [-1,-0.5] & 325 \\
 [-0.5,0]  & 382 \\
 [0,0.5]   & 392 \\
 [0.5,1]   & 308 \\
 [1,1.5]   & 155 \\
 [1.5,2]   & 86 \\
 [2,2.5]   & 32 \\
 [2.5,3]   & 6 \\
 [3,3.5]   & 2 \\
 \end{array}$$

In [1]:
using DataFrames ##Primero como buen R-Dependiente acomodamos los datos en un DataFrame
Frecuencias=[3, 6, 31, 80, 192, 325, 382, 392, 308, 155, 86, 32, 6, 2]
Infs=[-3.5+0.5(i) for i in 0:13]
datos=[Infs Infs+0.5 Frecuencias] 
Dat_datos=convert(DataFrame, datos)
rename!(Dat_datos, f => t for (f, t) = zip([:x1, :x2, :x3], [:Inferior, :Superior, :Frecuencia_abs]))

,Inferior,Superior,Frecuencia_abs
1,-3.5,-3.0,3.0
2,-3.0,-2.5,6.0
3,-2.5,-2.0,31.0
4,-2.0,-1.5,80.0
5,-1.5,-1.0,192.0
6,-1.0,-0.5,325.0
7,-0.5,0.0,382.0
8,0.0,0.5,392.0
9,0.5,1.0,308.0
10,1.0,1.5,155.0


### 1a

Dibuja el histograma de frecuencias relativas usando lo siguiente:

frecuencia relativa en intervalo = $\dfrac{\text{núm de observaciones en intervalo}}{\text{total de observaciones}}$

Altura de barra en intervalo = $\dfrac{\text{frecuencia relativa}}{\text{longitud de intervalo}}$

su histograma debe quedar como se muestra en la siguiente figura 

<img src="histograma_muestra.png">

In [2]:
#Primero obtenemos el total de observaciones:
Total_obs=sum(Dat_datos[:Frecuencia_abs])
##Añadimos la columna Frecuencia Relativa
Dat_datos[:Frecuencia_rel]=Dat_datos[:Frecuencia_abs]/Total_obs
Dat_datos

,Inferior,Superior,Frecuencia_abs,Frecuencia_rel
1,-3.5,-3.0,3.0,0.0015
2,-3.0,-2.5,6.0,0.003
3,-2.5,-2.0,31.0,0.0155
4,-2.0,-1.5,80.0,0.04
5,-1.5,-1.0,192.0,0.096
6,-1.0,-0.5,325.0,0.1625
7,-0.5,0.0,382.0,0.191
8,0.0,0.5,392.0,0.196
9,0.5,1.0,308.0,0.154
10,1.0,1.5,155.0,0.0775


In [65]:
##Como no nos interesa la altura para nuestra tabla(en el supuesto de que este documento fuera para fines laborales o de divulgación)
##Crearemos un arreglo con las alturas
Alturas=Dat_datos[:Frecuencia_rel]/0.5
using Interact, Plots
plotly()
plot(Dat_datos[:Superior], Alturas)

### 1b

Si el tamaño de los intervalos tiende a cero y tenemos sufientes muestras,entonces el histograma se aproxima a una campana gaussiana tal y como se muestra en la figura. <img src="campana_gaussiana.png">

Queremos aproximar la campana gaussiana mediante un spline cuadrático 
$$
S(x) =
\begin{cases}
 c_0(x), & x_0\leq x\leq x_1\\ 
  \vdots & \\
 c_{14}(x), & x_{14}\leq x\leq x_{15}\\  
\end{cases}
$$
de modo que 

> $\begin{array}{c} 
   \text{área bajo gráfica de }S(x) \\ \text{de nodo }i \text{ a nodo }i+1 
  \end{array}
  \quad = \quad
  \begin{array}{c} 
   \text{área de barra} \\ \text{de nodo }i \text{ a nodo }i+1
   \end{array}$

Para ello, el Spline en el intervalo $[x_i,x_{i+1}]$ se expresa como

$$\begin{array}{llll}
c_i(x) & = &   & S(x_i)(1-y_i)(1-3y_i) \\
       &   & + & S(x_{i+1})y_i(3y_i-2) \\
       &   & + & 6h_iy_i(1-y_i),
\end{array}$$

donde

$w_i$ es el ancho de barra en intervalo $[x_i,x_{i+1}]$

$h_i$ es la altura de barra en intervalo $[x_i,x_{i+1}]$

$y_i=\dfrac{x-x_i}{w_i}$



**Las incógnitas son $S(x_0),S(x_1),\dots,S(x_{14})$**

Tome $S'(x_0)=S'(x_{14})=0$. Use la continuidad de la derivada 
del spline en nodos interiores para formar el sistema de ecuaciones lineales

$T
\begin{pmatrix}
 S(x_0) \\ S(x_1) \\ \vdots \\ S(x_{13}) \\S(x_{14})
\end{pmatrix} =
\begin{pmatrix}
 3h_0 \\ 3/2(h_0+h_1) \\ \vdots \\ 3/2(h_{13}+h_{14}) \\ 3h_{14}
\end{pmatrix}, \qquad T\ \text{tridiagonal } 15\times 15$

Muestre la matriz $T$ usado comandos de Julia

¿puede usar la Factorización de cholesky para resolver el sistema? Explique su respuesta

### 1c

Resuelva el sistema de ecuaciones lineales por Factorización LU sin pivoteo, 

recuerde que $U=L^T$ en caso de poder usar cholesky con $l_{i,i}\neq 1$

### 1d 
Con los valores obtenidos construya en Julia una rutina para evaluar el spline cuadrático $S(x)$
en cualquier punto del intervalo $[x_0,x_{14}]$

### 1e

Gráfique el spline cuadrático junto con el histograma de frecuencias relativas

y la gaussiana $f(x)=\dfrac{\exp(-0.5x^2)}{\sqrt{2\pi}}$. *<font color=green>Use diferentes colores</font>*

## Ejercicio 2  $\quad$   <font color = blue>*El Cardioide*</font>

Damos los 2 arreglos

In [9]:
x = [ 1.0       
  1.29188   
  1.11095   
  0.439463  
 -0.439463  
 -1.11095   
 -1.29188   
 -1.0       
 -0.510053  
 -0.136026  
  0.00557906
  0.136026  
  0.510053  
  1.0     ];
y = [0.0        
  0.622139   
  1.39309    
  1.92541    
  1.92541    
  1.39309    
  0.622139   
  1.22465e-16
 -0.245629   
 -0.170571   
 -0.0244435  
 -0.170571   
 -0.245629   
 -2.44929e-16];
plot(x,y)

con los puntos $p_i=(x_i,y_i),\ i =0,\dots,13$ que mostramos en la siguiente figura.
<img src="cardioide.png">

Los puntos están ordenados comenzando con $(x_0,y_0)$ y los siguientes puntos 
$(x_i,y_i)$ se van colocando conforme avanzamos sobre 
la figura.

Queremos obtener una curva parametrizada

$$(x(l),y(l)),\quad 0\leq l\leq L$$

que pase por los puntos $(x_i,y_i)$ de la figura.

La idea es parametrizar la curva en función de su longitud $l$ desde
$(x_0,y_0)$ hasta el punto $(x,y)$ sobre la curva. En ese caso, 
la constante $L$ es la longitud de la curva desde $(x_{0},y_{0})$ hasta 
$(x_{13},y_{13})$.

Lo que hacemos es generar una partición del intervalo $[0,L]$:

$$0=l_0 <l_1<l_2<\cdots<l_{13}\approx L$$

a partir de las distancias 

$$d_i= \left\|
\begin{pmatrix}
 x_{i+1} \\ y_{i+1}
\end{pmatrix}-
\begin{pmatrix}
 x_i \\ y_i
\end{pmatrix}\right\|_2
\quad i=0,\dots,12$$

mediante la iteración 

$$l_{i+1} = l_i + d_i\quad i=0,\dots,12.$$

De ese modo,

> $x(l)\approx$ spline cúbico natural $S_x(l)$ que interpola
 $(l_0,x_0),\dots,(l_{13},x_{13})$ 
 
> $y(l)\approx$ spline cúbico natural $S_y(l)$ que interpola
 $(l_0,y_0),\dots,(l_{13},y_{13})$

### 2a

Gráfique el spline $S_x(l)$ sobre una partición
de $101$ puntos del intervalo $[l_0,l_{13}]$

### 2b
En otra figura, gráfique el spline $S_y(l)$ sobre una 
partición uniforme de $101$ puntos del intervalo $[l_0,l_{13}]$

### 2c

Gráfique los puntos  
 
 $$(S_x(l),S_y(l))$$ 
 
 usando una partición uniforme de $101$ puntos del intervalo 
 $l_0\leq l\leq l_{13}$.

### 2d
La longitud de la curva

  $$(S_x(l),S_y(l)),\quad l_0\leq l\leq l_{13}$$ 
  
  es 
  
  $$I = \int_{l_0}^{l_{13}}\sqrt{\left(\dfrac{d}{dl}S_x(l)\right)^2+
  \left(\dfrac{d}{dl}S_y(l)\right)^2} dl$$
  
  Calcule esta integral usando ***la regla de cuadratura compuesta del 
  trapecio*** con nodos iguales a $l_0,\dots,l_{13}$.

### 2e
Calcule 
$$L= \int_0^{2\pi}\sqrt{2+2\sin(\theta)}d\theta$$
por usando ***la regla de cuadratura compuesta del 
trapecio*** con 100 puntos.

¿cuánto vale el error $|I-L|$?

## Ejercicio 3  $\quad$   <font color = blue>*El Fenómeno de Runge*</font>

#### 2a
 
 Sea $p_k(t)$ el polinomio interpolante de grado $k$ para
 la función de Runge 
 
 $$f(t)=\dfrac{1}{t^2+1}:[-5,5]\to\mathbb R$$ 
 
 Haga una gráfica interactiva que muestre  lo siguiente:
 
 * la gráfica de $f(t)$ sobre la partición uniforme del 
 intervalo $[-5,5]$ de $100$ puntos
 
 * la gráfica de $p_k(t)$ usando la partición anterior
 
 * un deplazador que haga variar el grado $k$ del polinomio $p_k(t)$
 de $1$ a $10$

**NOTA:** Use diferencias divididas para obtener $p_k(t)$

In [90]:

f(t)=1/(t^2+1)

function P_k(f::Function,n::Number,x0::Number,xn::Number, x::Number)
    ##f es la función a interpolar, n el grado, x0 conce comienza el intervalo, xn donde termina, x el punto en el que se evaluará el polinomio
    ##Recordemos que n puntos nos dan un polinomio de grado n-1
    particion=linspace(x0,xn,n+1)
    valores_f=f.(particion)
    A=zeros(n+1,n+1)
    A[:,1]=valores_f
    for j in 2:n+1
        for i in j:n+1
            A[i,j]=(A[i,j-1]-A[i-1,j-1])/(particion[i]-particion[i-j+1])
        end
    end
    coeficientes=diag(A)
  
   
        poli=coeficientes[n+1]
           for k in 1:n
             poli=poli*(x-particion[n+1-k])+coeficientes[n+1-k]
           end
    
    return(poli)
end
function evaluapoli(f::Function, n::Number, x0::Number, xn::Number, m::Number)
    part=linspace(x0,xn,m)
    evaluaciones=zeros(m)
    for i in 1:m
        evaluaciones[i]=P_k(f,n,x0,xn, part[i])
    end
    return(evaluaciones)
end
intervalo=linspace(-5,5,100)
f_int=f.(intervalo)
@manipulate for Grado in 2:10
    P_n=evaluapoli(f,Grado,-5,5,100)
    plot(intervalo, P_n, xlabel="X", label="Polinomio grado $Grado", color="red")
    plot!(intervalo,f_int, label="Funcion", color="green")
end



Interact.Options{:SelectionSlider,Any}(77: "input-21" = 6 Any , "Grado", 6, "6", 5, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("2"=>2,"3"=>3,"4"=>4,"5"=>5,"6"=>6,"7"=>7,"8"=>8,"9"=>9,"10"=>10), Dict{Any,Any}(Pair{Any,Any}(7, "7"),Pair{Any,Any}(4, "4"),Pair{Any,Any}(9, "9"),Pair{Any,Any}(10, "10"),Pair{Any,Any}(2, "2"),Pair{Any,Any}(3, "3"),Pair{Any,Any}(5, "5"),Pair{Any,Any}(8, "8"),Pair{Any,Any}(6, "6"))), Any[], Any[], true, "horizontal")

## Ejercicio 4   $\quad$   <font color = blue>*Probabilidad de una Gaussina*</font>

Sea $$f(x)=\dfrac{\exp(-0.5x^2)}{\sqrt{2\pi}}$$

Queremos aproximar $$P(0\leq x\leq b) = \int_0^b f(x)dx$$

por cuadratura gaussiana de $n$ puntos

Elabore una rutina que reciba 
 * el número de nodos $n$
 * el límite de integración $b$

La  rutina debe devolver 
 * $I$ = la aproximación de $P(0\leq x\leq b)$
   por cuadratura gaussiana de $n$ puntos

**Nota:** Dentro de la rutina haga una válidación para que solo pueda
recibir los valores de $n=2,3,4$. En caso de que $n$ tenga otro valor,
Devuelva un mensaje de error.

## Ejercicio 5  $\quad$   <font color = blue>*Ecuación Integral*</font>

### 5a
Generamos una partición uniforme del intervalo $[-6,6]$ en $25$ puntos 
 
 $$-6=s_1<s_2<\dots <s_{25}=6.$$
 
 Sean $\omega_1,...,\omega_{25}$ los pesos de la regla de cuadratura 
 
 $$\int_{-6}^{\,6} k(t)dt=\sum_{i=1}^{25}\omega_j k(s_j).$$
 
¿Cuánto valen los pesos si usamos la cuadratura compuesta del trapecio?

¿Cuánto valen los pesos si usamos la cuadratura compuesta de Simpson?

### 5b 
Sean $k,g:[-6,6]\to\mathbb R$ las funciones dadas por

$$k(s) = 
 \begin{cases}
  1 + \cos\left(\dfrac{s\pi}{3}\right),  & \text{si $|s|<3$,} \\ 
  0, & \text{si $|s|\geq 3$,} 
 \end{cases}$$ 

$$g(s) = (6-|s|)\left(1+\dfrac{1}{2}\cos\left(\dfrac{s\pi}{3}\right)\right)
 + \dfrac{9}{2\pi}\sin\left(\dfrac{|s|\pi}{3}\right).$$
 
A partir de  los valores 
$g(s_1),\dots,g(s_{n+1})$, queremos calcular los valores 
$f(s_1),\dots,f(s_{n+1})$ de la función $f:[-6,6]\to\mathbb R$ que cumple 
la ecuación integral 

$$\int_{-6}^{\,6}k(s-t)f(t) dt = g(s).$$

Lo que hacemos es discretizar la ecuación integral en el sistema de ecuaciones lineales 

$$\begin{array}{cccc}
\begin{pmatrix}
 \omega_1k(s_1-s_1) & \omega_2k(s_1-s_2) & \dots & \omega_{25}k(s_1-s_{25}) \\
 \omega_1k(s_2-s_1) & \omega_2k(s_2-s_2) & \dots & \omega_{25}k(s_2-s_{25}) \\
 \vdots & \vdots & & \vdots \\
 \omega_1k(s_{25}-s_1) & \omega_2k(s_{25}-s_2) & \dots & \omega_{25}k(s_{25}-s_{25})
\end{pmatrix} &
\begin{pmatrix}
f(s_1) \\ f(s_2) \\ \vdots \\ f(s_{25})
\end{pmatrix} & = &
\begin{pmatrix}
g(s_1) \\ g(s_2) \\ \vdots \\ g(s_{25})
\end{pmatrix} \\
A & x & & b
\end{array}$$

mediante el método de colocación que usa los $26$ puntos $s_i$ y la regla de cuadratura de (a)

Usando los pesos para la regla de cuadratura de Simpson,
resuelva el sistema de ecuaciones lineales por factorización LU. 


Gráfique los puntos $(s_j,x_j)$ de la aproximación y los puntos $(s_j,k(s_j))$ de la solución $f(t)=k(t)$ en una misma figura. 

¿Hay diferencias notables entre las gráficas?

### 5c
 Ahora, usando los pesos para la regla de cuadratura del trapecio, 
 resuelva el sistema de  ecuaciones lineales 
 por factorización LU. Gráfique los puntos $(s_j,x_j)$ de  y $(s_j,k(s_j))$ 
 en una misma figura. 

¿Cúal de las dos reglas de cuadratura conviene usar?